In [ ]:
#SCRIPT63
import tkinter as tk
import random
import math

# ------------ Kelas Vector 2D Sederhana ------------
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    # Tambah vector lain
    def add(self, other):
        self.x += other.x
        self.y += other.y

    # Kalikan dengan skalar
    def mult(self, n):
        self.x *= n
        self.y *= n

    # Bagi dengan skalar
    def div(self, n):
        if n != 0:
            self.x /= n
            self.y /= n

    # Salin vector
    def copy(self):
        return Vector(self.x, self.y)

    # Panjang vector
    def mag(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

    # Normalisasi (jadikan panjang = 1)
    def normalize(self):
        m = self.mag()
        if m != 0:
            self.div(m)

    # Ubah magnitude (panjang)
    def set_mag(self, new_mag):
        self.normalize()
        self.mult(new_mag)


# ------------ Kelas Cairan (Liquid) ------------
class Liquid:
    def __init__(self, x, y, w, h, c):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.c = c  # Koefisien drag (resistance)

    # Cek apakah mover berada di dalam cairan
    def contains(self, mover):
        pos = mover.position
        return self.x < pos.x < self.x + self.w and self.y < pos.y < self.y + self.h

    # Hitung gaya hambat fluida (drag)
    def calculate_drag(self, mover):
        speed = mover.velocity.mag()
        drag_magnitude = self.c * speed ** 2

        # Arah drag berlawanan dengan arah kecepatan
        drag = mover.velocity.copy()
        drag.mult(-1)
        drag.set_mag(drag_magnitude)
        return drag

    # Gambar cairan di canvas
    def show(self, canvas):
        canvas.create_rectangle(self.x, self.y, self.x + self.w, self.y + self.h, fill="#ccf")


# ------------ Kelas Benda Bergerak (Mover) ------------
class Mover:
    def __init__(self, x, y, mass):
        self.mass = mass
        self.radius = mass * 8
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)

    # Terapkan gaya ke benda (F = m * a → a = F / m)
    def apply_force(self, force):
        f = force.copy()
        f.div(self.mass)
        self.acceleration.add(f)

    # Update posisi berdasarkan kecepatan dan percepatan
    def update(self):
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)  # Reset percepatan setelah digunakan

    # Gambar benda di canvas
    def show(self, canvas):
        x = self.position.x
        y = self.position.y
        r = self.radius
        canvas.create_oval(x - r, y - r, x + r, y + r, fill="gray", outline="black")

    # Cek apakah menyentuh lantai, jika ya, pantul
    def check_edges(self, height):
        if self.position.y > height - self.radius:
            self.velocity.y *= -0.9
            self.position.y = height - self.radius


# ------------ Program Simulasi ------------
class Simulation:
    def __init__(self, root):
        self.root = root
        self.width = 640
        self.height = 360
        self.canvas = tk.Canvas(root, width=self.width, height=self.height)
        self.canvas.pack()
        self.movers = []
        self.liquid = Liquid(0, self.height // 2, self.width, self.height // 2, 0.1)
        self.reset()

        self.root.bind("<Button-1>", self.mouse_pressed)
        self.draw()

    # Buat ulang objek mover secara acak
    def reset(self):
        self.movers = []
        for i in range(9):
            x = 40 + i * 60
            m = random.uniform(0.5, 3)
            self.movers.append(Mover(x, 0, m))

    # Jika mouse diklik, ulang simulasi
    def mouse_pressed(self, event):
        self.reset()

    # Fungsi utama untuk menggambar setiap frame
    def draw(self):
        self.canvas.delete("all")  # Hapus semua gambar lama
        self.liquid.show(self.canvas)  # Gambar cairan

        for mover in self.movers:
            # Jika mover masuk cairan, terapkan gaya drag
            if self.liquid.contains(mover):
                drag = self.liquid.calculate_drag(mover)
                mover.apply_force(drag)

            # Gaya gravitasi ke bawah (F = m * g)
            gravity = Vector(0, 0.1 * mover.mass)
            mover.apply_force(gravity)

            mover.update()
            mover.check_edges(self.height)
            mover.show(self.canvas)

        # Jalankan ulang fungsi draw tiap 16 ms (sekitar 60 fps)
        self.root.after(16, self.draw)


# ------------ Jalankan Program ------------
root = tk.Tk()
root.title("Simulasi Fluid Resistance (Sederhana)")
app = Simulation(root)
root.mainloop()


In [2]:
import tkinter as tk
import random

# ========== SETUP AWAL ==========
lebar, tinggi = 600, 400
gravitasi = 0.1
koefisien_air = 0.05

# ========== KELAS AIR ==========
class Air:
    def __init__(self):
        self.permukaan_air = tinggi // 2  # Ketinggian air
    
    def gambar(self, canvas):
        canvas.create_rectangle(0, self.permukaan_air, 
                              lebar, tinggi, 
                              fill="lightblue", outline="")
    
    def ada_di_air(self, y_benda):
        return y_benda > self.permukaan_air

# ========== KELAS BENDA ==========
class Benda:
    def __init__(self, x, massa):
        self.x = x
        self.y = 0
        self.massa = massa
        self.kecepatan = 0
        self.ukuran = massa * 10
    
    def update(self):
        # Hitung percepatan (F = m*a -> a = F/m)
        gaya_gravitasi = self.massa * gravitasi
        percepatan = gaya_gravitasi / self.massa
        
        # Jika di air, tambahkan hambatan
        if air.ada_di_air(self.y):
            percepatan -= koefisien_air * self.kecepatan**2
        
        # Update kecepatan dan posisi
        self.kecepatan += percepatan
        self.y += self.kecepatan
        
        # Cek jika menyentuh dasar
        if self.y > tinggi - self.ukuran:
            self.y = tinggi - self.ukuran
            self.kecepatan *= -0.8  # Memantul
    
    def gambar(self, canvas):
        canvas.create_oval(self.x - self.ukuran, self.y - self.ukuran,
                          self.x + self.ukuran, self.y + self.ukuran,
                          fill="blue")

# ========== PROGRAM UTAMA ==========
def buat_benda_baru():
    semua_benda.clear()
    for i in range(5):  # Buat 5 benda acak
        massa_acak = random.uniform(1, 3)
        x_pos = 100 + i * 100
        semua_benda.append(Benda(x_pos, massa_acak))

def update_simulasi():
    canvas.delete("all")
    air.gambar(canvas)
    
    for benda in semua_benda:
        benda.update()
        benda.gambar(canvas)
    
    root.after(30, update_simulasi)  # Update setiap 30ms

# ========== JALANKAN PROGRAM ==========
root = tk.Tk()
root.title("Simulasi Benda Jatuh")
canvas = tk.Canvas(root, width=lebar, height=tinggi, bg="white")
canvas.pack()

# Buat objek air dan daftar benda
air = Air()
semua_benda = []
buat_benda_baru()

# Klik untuk reset
canvas.bind("<Button-1>", lambda event: buat_benda_baru())

# Mulai simulasi
update_simulasi()
root.mainloop()